# Preprocesamiento de la base de datos

## Contenido

1. [Mapeo de los datos](#Mapeo-de-los-datos)
2. [Simplificación de variables categóricas](#Simplificación-de-variables-categóricas)
3. [Eliminación de datos fuertemente correlacionados](#Eliminación-de-datos-fuertemente-correlacionados)
4. [Balanceamiento de los datos](#Balanceamiento-de-los-datos)

In [1]:
# Importación
import sys 
import os
from warnings import filterwarnings
from pandas import read_csv, concat, DataFrame
from pickle import load

proyect = os.path.abspath('../..')
if proyect not in sys.path:
    sys.path.append(proyect)
from Preprocessing.cleaning import chisq_matrix

filterwarnings('ignore')

## Mapeo de los datos

La base de datos original se encuentra en formato csv. A continuación se puede ver la primera columna de la variable.

In [2]:
data = read_csv('HMDA.csv')
print(f'Dimensión de la base de datos: {data.shape}')
data.head()

Dimensión de la base de datos: (387491, 99)


,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2022,5493000YNV8IX4VD3X12,35614,NY,36005.0,3.600505e+10,C,Conventional:First Lien,Single Family (1-4 Units):Manufactured,Not Hispanic or Latino,...,NaN,NaN,NaN,2142,32.31,99000,109,339,411,70
1,2022,5493006O6Q2F0MR0ZM22,0,NY,NaN,NaN,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,0,0.00,0,0,0,0,0
2,2022,KD3XUN7C6T14HNAYLU02,35614,NY,36061.0,3.606101e+10,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,6180,19.71,99000,292,711,685,0
3,2022,KD3XUN7C6T14HNAYLU02,35614,NY,36061.0,3.606100e+10,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,2989,57.51,99000,284,208,7,19
4,2022,KD3XUN7C6T14HNAYLU02,35614,NY,36061.0,3.606102e+10,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6659,22.65,99000,292,1392,267,0


Previamente se hizo un re-procesamiento a la base de datos, eliminando columnas repetidas, identificadores específicos para cada préstamo, y re-distribución de algunas variables que actuaron como cache para más de una categoría, como ocurre con es *AUS*, que puede presentar distintos sistemas y se almacena en los *aus_* de 1 a 5. Por otro lado, se reemplazaron los valores numéricos mapeados en las columnas por los respectivos nombres mencionados en el glosario. Los datos finales se encuentran en la siguiente tabla.

In [3]:
data = read_csv('hmda_mapped.csv', dtype={'total_units':str} )
print(f'Dimensión de la base de datos mapeada: {data.shape}')
data.head()

Dimensión de la base de datos mapeada: (387491, 62)


,conforming_loan_limit,derived_ethnicity,derived_race,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,...,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus_DU,aus_LP,aus_TOTAL,aus_GUS,aus_other
0,C,Not Hispanic or Latino,Native Hawaiian or Other Pacific Islander,0,Not applicable,Preapproval not requested,Conventional,Home purchase,Secured by a first lien,Not a reverse mortgage,...,99000,109,339,411,70,0,0,0,0,0
1,NC,Not Hispanic or Latino,White,0,Not applicable,Preapproval not requested,Conventional,Cash-out refinancing,Secured by a first lien,Not a reverse mortgage,...,0,0,0,0,0,0,0,0,0,0
2,NC,Ethnicity Not Available,Race Not Available,1,Not applicable,Preapproval not requested,Conventional,Other,Secured by a first lien,Not a reverse mortgage,...,99000,292,711,685,0,0,0,0,0,0
3,NC,Ethnicity Not Available,Race Not Available,1,Not applicable,Preapproval not requested,Conventional,Other,Secured by a first lien,Not a reverse mortgage,...,99000,284,208,7,19,0,0,0,0,0
4,NC,Not Hispanic or Latino,White,1,Not applicable,Preapproval not requested,Conventional,Home purchase,Secured by a first lien,Not a reverse mortgage,...,99000,292,1392,267,0,0,0,0,0,0


## Simplificación de variables categóricas

Para mayor comodidad, todos los objetos relacionados al modelo fueron almacenados en archivos `.pkl`, en este caso, se dividió la base de datos en numéricas, categóricas y la variable respuesta. A continuación podemos ver los resultados de una prueba chi cuadrado entre las variables para descartar aquellas variables irrelevantes. El 1 marca un valor p inferior o igual al $\alpha (0.05)$ y el 0 lo contrario.

In [4]:
with open( '../../Preprocessing/data_raw.pkl', 'rb' ) as f:
    X, cat, y = load(f)

chisq = chisq_matrix( concat( ( DataFrame( y ), cat ), axis=1 ) )
chisq_res = chisq.map( lambda x: 1 if x <= 0.001 else 0 )
chisq_res

,action_taken,conforming_loan_limit,derived_ethnicity,derived_race,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,...,co-applicant_age,submission_of_application,initially_payable_to_institution,applicant_ethnicity_observed,co-applicant_ethnicity_observed,aus_DU,aus_LP,aus_TOTAL,aus_GUS,aus_other
action_taken,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
conforming_loan_limit,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
derived_ethnicity,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
derived_race,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
purchaser_type,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
preapproval,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
loan_type,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
loan_purpose,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
lien_status,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
reverse_mortgage,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [5]:
chisq_res.sum()[ chisq_res.sum() < 41 ]

loan_type                                   40
negative_amortization                       40
other_nonamortizing_features                40
construction_method                         38
manufactured_home_land_property_interest    40
aus_DU                                      40
aus_LP                                      40
aus_TOTAL                                   40
aus_GUS                                     37
dtype: int64

Finalmente, se pueden descartar aquellas variables que estén fuertemente relacionadas con otras. Por otro lado, es notorio que todas tienen una fuerte relación con la variable respuesta. Teniendo en cuenta todo esto, podemos selecionar las variables que no se correlacionen entre sí, en este caso, son `construction_method` y `aus_GUS`.

## Eliminación de datos fuertemente correlacionados

La eliminación se lleva a cabo por medio de una prueba de VIF, calculando todos los VIFs entre sí y eliminando la variable con mayor inflación en caso que esta supere el valor de 10. Por otro lado, las variables categóricas seleccionadas en el paso anterior son encodificadas mediante un `OneHotEncoder` y sustituídas en la base de datos original, de modo que la evaluación de los modelos sea más sencilla. La base de datos para el entrenamiento finalmente es la siguiente:

In [6]:
with open( '../../Preprocessing/data_red.pkl', 'rb' ) as f:
    X, y = load(f)

X.head()

,loan_amount,loan_to_value_ratio,loan_term,property_value,income,ffiec_msa_md_median_family_income,construction_method_Manufactured home,construction_method_Site-built,aus_GUS_0,aus_GUS_1
0,125000.0,80.000,300.0,125000.0,39.0,99000,1.0,0.0,1.0,0.0
1,1585000.0,80.000,360.0,2275000.0,100.0,0,0.0,1.0,1.0,0.0
2,5805000.0,80.000,360.0,7255000.0,262.0,99000,0.0,1.0,1.0,0.0
3,2245000.0,70.165,360.0,3195000.0,663.0,99000,0.0,1.0,1.0,0.0
4,2175000.0,71.148,360.0,3055000.0,396.0,99000,0.0,1.0,1.0,0.0


Podemos ver que de las 61 variables que se tenían originalmente, quedan únicamente 10.

## Balanceamiento de los datos

En última instancia, se tomará un dataset balanceado por medio de Tomek Links y un undersampling aleatoreo de modo que se reduzcan el número de observaciones de la clase mayoritaria. Cabe resaltar que Tomek Links no elimina un porcentaje de los datos, sino aquellos datos de la clase mayoritaria que se encuentren extremadamente cercanos a datos de la clase minoritaria, de modo que causen ruido en los datos. Por tanto, se requiere de otras técnicas de undersampling para balancear efectivamente los datos. Este paso es posterior a la partición de entrenamiento y prueba, de modo que ambas clases puedan ser calificadas con el mismo conjunto. Finalmente, se toma el set de entrenamiento base y el balanceado, de modo que obtenemos:

In [7]:
with open( '../../Models/data_models.pkl', 'rb' ) as f:
    data = load(f)

print( f'Tamaño del set de entrenamiento base: { data["base"][0].shape }' )
print( f'Tamaño del set de entrenamiento balanceado: { data["balanced"][0].shape }' )
print( f'Tamaño del set de prueba: { data["train"][0].shape }' )


Tamaño del set de entrenamiento base: (309992, 10)
Tamaño del set de entrenamiento balanceado: (146000, 10)
Tamaño del set de prueba: (77499, 10)


Cabe aclarar que el nombre *train*  es el correspondiente al set de prueba, a pesar que paresca lo contrario.